In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, LSTM
from keras.models import Sequential
import matplotlib.pyplot as plt

In [ ]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv"
df_temp = pd.read_csv(url, parse_dates=['Date'], index_col='Date')
df_temp

In [ ]:
scaler = MinMaxScaler(feature_range = (0,1))
df_temp['Temp'] = scaler.fit_transform(df_temp[['Temp']])
df_temp

In [ ]:
def crear_secuencia(data, time_steps):
    X, y = [], []
    for i in range (len(data) - time_steps):
        X.append(data[i:(i+time_steps)])
        y.append(data[i+time_steps])
    return